In [1]:
import pyspark
import math

In [2]:
sc = pyspark.SparkContext(appName = "teste")
sc

<SparkContext master=local[*] appName=teste>

In [3]:
rdd = sc.sequenceFile("part-00000")

In [23]:
N = rdd.count()
N

36133

In [8]:
def conta_palavras(item):
    url, conteudo = item 
    palavras = conteudo.strip().split()
    return [(palavra, 1) for palavra in palavras]

# rdd.flatMap(conta_palavras)

def junta_contagens(nova_contagem, contagem_atual):
    return nova_contagem + contagem_atual

rdd_word_freq = rdd.flatMap(conta_palavras).reduceByKey(junta_contagens)

In [29]:
M = rdd_word_freq.count()
M

1656657

In [9]:
def conta_documento(item):
    url, conteudo = item 
    palavras = conteudo.strip().split()
    return [(palavra, 1) for palavra in set(palavras)]

rdd_doc_freq = rdd.flatMap(conta_documento).reduceByKey(junta_contagens)

In [18]:
doc_min = 5
doc_max = 0.7* N
def filtra_doc_freq(item):
    contagem = item[1]
    return (contagem < doc_max) and (contagem >= doc_min)


rdd_doc_freq_filtrado = rdd_doc_freq.filter(filtra_doc_freq)

In [13]:
rdd_doc_freq_filtrado.count()

256108

In [30]:
def computa_idf(item):
    palavra, contagem = item
    idf = math.log10(N / contagem)
    return (palavra,idf)

def computa_freq(item):
    palavra, contagem = item
    freq = math.log10(1 + contagem)
    return (palavra,freq)

rdd_idf = rdd_doc_freq_filtrado.map(computa_idf)
rdd_freq = rdd_word_freq.map(computa_freq)
    

In [32]:
#Nao funciona no cluster
result = rdd_idf.collect()
result1 = rdd_freq.collect()

In [33]:
test = sorted(result, key = lambda x: x[1])
test1 = sorted(result1, key = lambda x: x[1])

In [34]:
def idf_palavra(lista, palavra):
    for e in range(0,len(lista)):
        for p in test[e]:
            if p == palavra:
                return lista[e][1]

In [36]:
idf_palavra(test, "Notícias")

0.6015914902163717